In [12]:
#Instalamos la librería que vamos a necesitar para abrir el archivo dta
!pip install pyreadstat
# Primero importamos las librerías necesarias para leer y manipular la base de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyreadstat as prs
%pip install pyod 
# Primero instalamos las librerías necesarias para leer y manipular la base de datos
%pip install pyod 
#Este paquete lo vamos a usar para calcular el MAD, lo cual va a ser necesario para eliminar los outliers.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
# Abrimos la carpeta donde se encuentran las bases de datos
os.chdir(r"C:/Users\s1248850/OneDrive - Syngenta/Documents/Rosario Luque/Ciencias de datos/TP/TP3")
# Guardamos las bases de datos en en dos variables
ind_2004, meta = prs.read_dta("Individual_t104.dta")
%pip install openpyxl
ind_2024 = pd.read_excel("usu_individual_T124.xlsx")

Note: you may need to restart the kernel to use updated packages.


a. Eliminen todas las observaciones que no corresponden a los
aglomerados de Ciudad Autónoma de Buenos Aires o Gran Buenos
Aires, y unan ambos trimestres en una sola base.

In [30]:
ind_2024['AGLOMERADO']

172       2
173       2
174       2
175       2
176       2
         ..
46036    32
46037    32
46038    32
46039    32
46040    32
Name: AGLOMERADO, Length: 2462, dtype: int64

In [27]:
# Filtramos los aglomerados de Ciudad Autónoma de Buenos Aires (2) y Gran Buenos Aires (32)
ind_2004 = ind_2004[ind_2004['aglomerado'].isin([2, 32])]
ind_2024 = ind_2024[ind_2024['AGLOMERADO'].isin([2, 32])]

# Unimos ambas bases de datos
df_unificado = pd.concat([ind_2004, ind_2024], ignore_index=True)
df_unificado.head()

,CODUSU,nro_hogar,componente,h15,ano4,trimestre,region,mas_500,aglomerado,pondera,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
0,125098,1.0,1.0,1.0,2004.0,1.0,1.0,S,32.0,1299.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,125397,1.0,1.0,1.0,2004.0,1.0,1.0,S,32.0,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,125515,1.0,1.0,1.0,2004.0,1.0,1.0,S,32.0,1486.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,125558,1.0,1.0,1.0,2004.0,1.0,1.0,S,32.0,1674.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,125558,1.0,2.0,1.0,2004.0,1.0,1.0,S,32.0,1674.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


b. Si hay observaciones con valores que no tienen sentido,
descártenlas (por ejemplo, ingresos y edades negativos). Expliquen
las decisiones tomadas.